# OCTIS Model Evaluation

## Prologue & Imports

We will evaluate the performance of most relevant OCTIS models as a baseline for non-SOTA Topic Modeling. These models will be compared on the same preprocessed dataset, the same number of topics and the same evaluation metrics.

In [4]:
from octis.models import LDA, ETM, NMF, NeuralLDA
from octis.dataset.dataset import Dataset
from octis.evaluation_metrics.diversity_metrics import TopicDiversity
from octis.evaluation_metrics.coherence_metrics import Coherence
from spacy.lang.el.stop_words import STOP_WORDS as el_stop
from spacy.lang.en.stop_words import STOP_WORDS as en_stop

from utils.data_loader import GreekPMDataloader
from models.octis.utils.gr_preprocessor import GreekStanzaPreprocessor

import pandas as pd

ImportError: attempted relative import with no known parent package

## Dataset Loading

If our dataset has already been processed and cached, then we can load it. Otherwise, we will preprocess it and save it for future use.

In [3]:
try:
    dataset = Dataset()
    dataset.load_data_from_csv('data/octis/dataset_processed')
except:
    # Merge data and prepare for preprocessing
    try:
        speeches_df = pd.read_csv('data/data_speeches.csv')
        statements_df = pd.read_csv('data/data_statements.csv')
    except: 
        ds = GreekPMDataloader() # If the data is not available, download it
        cats_df = ds.load_categories("speeches", "statements")

    df = pd.concat([speeches_df, statements_df], ignore_index=True)
    
    # Drop irrelevant columns and convert to string
    df['text'] = df['text'].astype(str)
    df = df.drop(columns=['date', 'id', 'url', 'title']).dropna(how='any')
    
    df.to_csv('data/data_merged.csv', index=False)

    # We have some non-Greek stopwords in the dataset, so we need to remove them
    stopwords = set(el_stop).union(set(en_stop))
    
    # Initialize preprocessing
    preprocessor = GreekStanzaPreprocessor(
                             stopword_list=stopwords,
                             remove_punctuation=True, remove_numbers=True,
                             max_df=0.2, min_df=0.01, min_chars=4, min_words=20, 
                             num_processes=6)
    
    # Create the dataset
    dataset = preprocessor.preprocess_dataset(documents_path='data/data_merged.csv')
    
    dataset.save('data/octis/dataset_processed')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2032 entries, 0 to 2031
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    2032 non-null   object
 1   id      2032 non-null   int64 
 2   url     2032 non-null   object
 3   title   2032 non-null   object
 4   text    2012 non-null   object
dtypes: int64(1), object(4)
memory usage: 79.5+ KB


## Model Initialization

In [13]:
model = NeuralLDA(num_topics=13, num_neurons=200, num_layers=2)

In [14]:
output = model.train_model(dataset)

Epoch: [1/100]	Samples: [1442/144200]	Train Loss: 3599.828114164355	Time: 0:00:00.114037
Epoch: [1/100]	Samples: [161/16100]	Validation Loss: 153238703.3591809	Time: 0:00:00.004465
Epoch: [2/100]	Samples: [2884/144200]	Train Loss: 3666.4920087118585	Time: 0:00:00.115254
Epoch: [2/100]	Samples: [161/16100]	Validation Loss: 45624.85326086957	Time: 0:00:00.004576
Epoch: [3/100]	Samples: [4326/144200]	Train Loss: 3395.3487235610264	Time: 0:00:00.109990
Epoch: [3/100]	Samples: [161/16100]	Validation Loss: 3201.4151785714284	Time: 0:00:00.004606
Epoch: [4/100]	Samples: [5768/144200]	Train Loss: 3372.738015776699	Time: 0:00:00.105471
Epoch: [4/100]	Samples: [161/16100]	Validation Loss: 3165.120875388199	Time: 0:00:00.004396
Epoch: [5/100]	Samples: [7210/144200]	Train Loss: 3384.5410996012483	Time: 0:00:00.105965
Epoch: [5/100]	Samples: [161/16100]	Validation Loss: 3138.5403726708073	Time: 0:00:00.004676
Epoch: [6/100]	Samples: [8652/144200]	Train Loss: 3338.6350825676145	Time: 0:00:00.102723


In [15]:
output.keys()

dict_keys(['topics', 'topic-document-matrix', 'topic-word-matrix', 'test-topic-document-matrix'])

In [16]:
for t in output['topics']:
  print(" ".join(t))

εκλογή μνημόνιο χιλιάδα θυσία φόρος ανεργία λάθος πλεόνασμα ψηφίζω αντιπολίτευση
υφυπουργός σύσκεψη εκτιμώ παραπληροφόρηση ταχύτητα ξεχωριστός γρήγορος επίπτωση καλοκαίρι εσωτερικά
δημοσιονομικός οργανισμός έτοιμος συμμετοχή επανέρχομαι πρόσβαση παρέχω διαμορφώνω ενεργειακός ερώτηση
χέρι οργάνωση μηχανισμός συγκεκριμένα μοναδικός εύκολος χρονικός πρακτικός χρηματοδοτώ ικανός
εξαιρετικός παραπληροφόρηση υποδέχομαι συγχαρητήριο λαμβάνω γεωπολιτικός καταστροφικά μετάβαση δημιουργικότητα συνομιλώ
τουρισμός παραγωγή παραγωγικός πανεπιστήμιο δημοκρατικός λάθος δικαιοσύνη τεχνολογία διαπραγμάτευση βουλευτής
δολοφονία υπεραξία μνημόνια μητροπολίτης βρετανία σλοβενία βέλγιο χριστούγεννα θράκη διατίθεμαι
that also have with much will about very τεχνολογία εμπειρία
προσωπικό νοσοκομείο μονάδα νομοσχέδιο δικαιοσύνη διαφορά δρομολογώ κρίνω αποκτάω αίθουσα
παραγωγή προϋπόθεση ανταγωνιστικότητα τρίτος δίκτυο δουλεύω αγροτικός περιμένω προϋπολογισμός δισεκατομμύριο
μετατρέπω σύνολο περιγράφω κομμάτι π

In [17]:
npmi = Coherence(texts=dataset.get_corpus(), topk=5, measure='c_npmi')
cv = Coherence(texts=dataset.get_corpus(), topk=5, measure='c_v')
umass = Coherence(texts=dataset.get_corpus(), topk=5, measure='u_mass')
uci = Coherence(texts=dataset.get_corpus(), topk=5, measure='c_uci')

In [18]:
topic_diversity = TopicDiversity(topk=5)

In [19]:
topic_diversity_score = topic_diversity.score(output)
print("Topic diversity: "+str(topic_diversity_score))

npmi_score = npmi.score(output)
print("NPMI Coherence: "+str(npmi_score)) 

cv_score = cv.score(output)
print("C_V Coherence: "+str(cv_score))

umass_score = umass.score(output)
print("U_MASS Coherence: "+str(umass_score))

uci_score = uci.score(output)
print("C_UCI Coherence: "+str(uci_score))

Topic diversity: 0.9692307692307692
NPMI Coherence: -0.06526148338418904
C_V Coherence: 0.5432437733522696
U_MASS Coherence: -3.643577635249891
C_UCI Coherence: -2.798853319611031
